In [ ]:
import spimage
import h5py
import numpy as np
import scipy.ndimage as ndimage
import scipy.ndimage.morphology as morphology
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
filename = "/Users/benedikt/data/LCLS/amol3416/pbcv/phasing/r0098/r0098_%03d.cxi"

In [ ]:
pixelsize  = 4*75*1e-6
distance   = 0.37
wavelength = 1.59e-9
shape = (256,256)
x_to_q = lambda x: 2*np.sin(np.arctan2(x*pixelsize, distance)/2.) / wavelength
dx = wavelength * distance / (shape[1] * pixelsize) * 1e9 #[nm/px]

In [ ]:
thresholds = [0.04, 0.04, 0.05, 0.045, 0.04, 0.04, 0.04, 0.04, 0.04, 0.05]
selections = []
for i in range(10):
    with h5py.File(filename %i, 'r') as f:
        real_error = f['scores_final']['real_error'][:]
        selections.append(real_error < thresholds[i])

In [ ]:
output = []
for i in range(10):
    with h5py.File(filename %i, 'r') as f:
        real_space_final = f['real_space_final'][selections[i],:,:]
        support_final    = f['support_final'][selections[i],:,:]
        output.append(spimage.prtf(real_space_final, support_final, translate=True, enantio=True))

In [ ]:
# Resulst from PRTF calculation
images = []
for i in range(10):
    images.append(output[i]['super_image']/selections[i].sum())

In [ ]:
c = 110
fig, axes = plt.subplots(nrows=2, ncols=5)
fig.set_size_inches(10, 4, forward=True)
for i in range(10):
    k,l = i//5,i%5
    im = axes[k,l].imshow(np.abs(images[i])[c+1:-c,c+1:-c], cmap='viridis', vmin=0.01, vmax=2.2, interpolation='none')
    axes[k,l].axis('off')
    axes[k,l].text(0,0, '%d/1000' %selections[i].sum(),  color='w', va='top', ha='left')
axes[1,4].add_patch(plt.Rectangle((2,32),100./7.7,1.2, facecolor='w', lw=0))
axes[1,4].text(8.3,31.5, r'100 nm', color='w', weight='bold', fontsize=10, ha='center', va='bottom')
ax = fig.add_axes((.9,0.127,0.02,0.775))
cb = fig.colorbar(im, cax=ax)
cb.outline.set_visible(False)
cb.ax.set_ylabel('Electron density [arb. units]')
plt.subplots_adjust(wspace=0.001, hspace=0.01)
plt.show()